# Landsat Satellite Data Processing
The following Jupyter Notebook details the extraction of Landsat 8 satellite imagery data and construction of the satellite dataset - one of the two datasets necessary for the MAPWAPS project application.

Note: This spearate Jupyter Notebook was created to ensure proper code operation and limited code error (i.e. get the code to work how it needs to) before combining it with the flux tower data as done in the Data Aquisition, Processing and Collation.ipynb Jupyter Notebook.


## Library and Function Imports

This cell imports several essential libraries and sets up functionalities in the Jupyter Notebook, ensuring that they are readily available for implementation and utilization later in the code

In [2]:
!pip install rasterio
import ee
import os
import rasterio
import numpy as np
import pandas as pd
import geemap
import re
from IPython.display import Image, display

# Authenticate with Earth Engine (requires user interaction)
ee.Authenticate()

# Initialize the Earth Engine Python API
ee.Initialize()

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 32.2 MB/s eta 0:00:00
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=fXgvwu6atjBGUsOYyupO5ROdJDvVgGYuilBggm1m99s&tc=cIz3ZtFt212g4wdLwsWMh3FAZXoFNED0sokmmB3J_7s&cc=g46_yC-pJNe1UlPHd6y_zbj_p7i7_CR4xU2_CcJYuFw

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXlmoFaKq7QMJyCs9RXaEfBRZsa-dEXwuhXmxCrnQk0mmjpXGHqrdvo

Successfully saved authorization token.
Mounted at /content/drive


## Miscellaneous Functions

### save_df_to_drive
Function that saves a Pandas DataFrame as a csv file to a Google Drive folder specified by a google drive file path

In [3]:
def save_df_to_drive(df, file_path_in_google_drive):
  """
  Function that saves a Pandas DataFrame as a csv file to a Google Drive folder specified by a google drive file path

    parameter:  dataframe -> the Pandas DataFrame needing to be saved
                file_path_in_drive -> Google Drive folder file path that will store the csv file
    return:     void

  """
  try:
      # Ensure the destination directory exists
      destination_dir = os.path.dirname(file_path_in_google_drive)
      os.makedirs(destination_dir, exist_ok=True)

      # Save the DataFrame to the specified file path in Google Drive
      df.to_csv(file_path_in_google_drive, index=False)

      print(f"DataFrame saved to Google Drive at '{file_path_in_google_drive}'")

  except Exception as e:
      print("An error occurred:", str(e))

## Single Image
All code was first developed for a single image before extrapolating it to larger collections and datasets

### get_single_landsat_image_id
Function that extracts a single Landsat image ID from a collection specified by a date range and location (specified as co-ordinate)

In [7]:
def get_single_landsat_image_id(latitude, longitude, start_date, end_date):
  """
  Function that extracts a single Landsat image ID from a collection specified by a date range and location (specified as co-ordinate)

    parameter:  start_date -> start date of the date range
                end_date -> end date of the date range
                latitude -> latitude co-ordinate of desired location
                longitude -> longitude co-ordinate of desired location
    return:     image_id -> Landsat 8 image ID

  """

  # Create a geometry point for the specified latitude and longitude
  point = ee.Geometry.Point(longitude, latitude)

  # Create a Landsat image collection
  landsat_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
      .filterBounds(point) \
      .filterDate(start_date, end_date) \
      .sort('CLOUD_COVER')

  # Get the first (least cloudy) image in the collection
  image = ee.Image(landsat_collection.first())

  # Get the image ID
  image_id = image.id().getInfo()

  return image_id

# -----------------------------------------------------------------------------------------------------------------
# Example usage
start_date = '2021-01-01'
end_date = '2021-12-31'
latitude = 37.7749
longitude = -122.4194

landsat_image_id = get_single_landsat_image_id(latitude, longitude, start_date, end_date)
print("Landsat Image ID:", landsat_image_id)


Landsat Image ID: LC08_044034_20210321


### display_interactive_landsat_image
Function that displays an interactive Landsat 8 image specified by its image ID

In [9]:
def display_interactive_landsat_image(image_id):
  """
  Function that displays an interactive Landsat 8 image specified by its image ID

    parameter:  image_id -> Landsat 8 image ID
    return:     void -> interactive display

  """
  try:
      # Load the Landsat image by its ID
      landsat_image = ee.Image(image_id)

      # Create an interactive map
      Map = geemap.Map(center=[37.7749, -122.4194], zoom=10)

      # Add the Landsat image to the map
      Map.addLayer(landsat_image, {
          'bands': ['B4', 'B3', 'B2'],  # Display Red, Green, and Blue bands
          'min': 0,
          'max': 0.3  # Adjust the min-max values as needed for visualization
      }, 'Landsat Image')

      # Display the map
      Map
      display(Map)

  except Exception as e:
      print("An error occurred:", str(e))

# -----------------------------------------------------------------------------------------------------------------
# Example usage:
landsat_image_id = 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210321'  # Replace with your Landsat image ID
display_interactive_landsat_image(landsat_image_id)


Map(center=[37.7749, -122.4194], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

### display_static_landsat_image
Function that displays a static Landsat 8 image specified by its image ID

In [10]:
def display_static_landsat_image(image_id):
  """
  Function that displays a static Landsat 8 image specified by its image ID

    parameter:  image_id -> Landsat 8 image ID
    return:     void -> static display

  """
  try:
      # Load the Landsat image by its ID
      landsat_image = ee.Image(image_id)

      # Generate a URL for the image thumbnail with dimensions
      image_url = landsat_image.getThumbURL({
          'bands': ['B4', 'B3', 'B2'],  # Display Red, Green, and Blue bands
          'min': 0,
          'max': 0.3,  # Adjust the min-max values as needed for visualization
          'format': 'jpg',  # You can choose other formats like 'png'
          'dimensions': 800  # Adjust the dimensions of the thumbnail image
      })

      # Display the image using IPython.display
      display(Image(url=image_url))

  except Exception as e:
      print("An error occurred:", str(e))

# -----------------------------------------------------------------------------------------------------------------
# Example usage:
landsat_image_id = 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210321'  # Replace with your Landsat image ID
display_static_landsat_image(landsat_image_id)


## Plant Indice Calculations


### get_landsat_bands
Function that extracts the multispectral bands from a Landsat 8 satellite image of a co-ordinate specified location

In [21]:
def get_landsat_bands(image_id, latitude, longitude):
  """
  Function that extracts the multispectral bands from a Landsat 8 satellite image of a co-ordinate specified location

    parameter:  image_id -> unique Landsat 8 image ID
                latitude -> latitude co-ordinate of desired location
                longitude -> longitude co-ordinate of desired location
    return:     band_dict -> dictionary of multispectral band values for a specified location with labels B1 - B11

  """
  try:
      # Load the Landsat image by its ID
      landsat_image = ee.Image(image_id)

      # Define a point geometry for the specified latitude and longitude
      point_geometry = ee.Geometry.Point([longitude, latitude])

      # Use the .sample() method to extract pixel values at the specified geometry
      # This will create a feature collection containing the pixel values
      pixel_values = landsat_image.sample(point_geometry, 30)  # 30 meters scale for Landsat

      # Initialize an empty dictionary to store the band values
      band_dict = {}

      # Extract band values from the feature collection and add them to the dictionary
      for band_name in landsat_image.bandNames().getInfo():
          band_value = pixel_values.first().get(band_name).getInfo()
          band_dict[band_name] = band_value

      return band_dict

  except ee.EEException as e:
      return {"error": "An Earth Engine exception occurred: " + str(e)}
  except Exception as e:
      return {"error": "An unexpected error occurred: " + str(e)}

# -----------------------------------------------------------------------------------------------------------------
# Example usage:
landsat_image_id = 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210321'
latitude = 37.421
longitude = -122.081

band_data = get_landsat_bands(landsat_image_id, latitude, longitude)
print(band_data)

{'B1': 0.1784408837556839, 'B2': 0.1667034775018692, 'B3': 0.15561515092849731, 'B4': 0.16572986543178558, 'B5': 0.23493723571300507, 'B6': 0.20380879938602448, 'B7': 0.16992180049419403, 'B8': 0.15285660326480865, 'B9': 0.0019472124986350536, 'B10': 290.8738708496094, 'B11': 290.0387878417969, 'BQA': 2752}


### get_cloud_cover
Function that extracts the cloud cover percentage from a satellite image specified by a Landsat 8 image ID


In [22]:
def get_cloud_cover(image_id):
  """
  Function that extracts the cloud cover percentage from a satellite image specified by a Landsat 8 image ID

    parameter:  image_id -> Landsat 8 image ID
    return:     cloud_cover_rounded -> percentage cloud cover rounded to 3 decimal points

  """
  # Load the Landsat image using the provided image_id
  image = ee.Image(image_id)

  # Get the cloud cover property
  cloud_cover = image.get('CLOUD_COVER').getInfo()

  # Round the cloud cover percentage to 3 decimal places
  cloud_cover_rounded = round(cloud_cover, 3)

  return cloud_cover_rounded

# -----------------------------------------------------------------------------------------------------------------
# Example usage:
landsat_image_id = 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210116'
cloud_cover = get_cloud_cover(landsat_image_id)
print(f'Cloud Cover: {cloud_cover}%')


Cloud Cover: 25.79%


### calculate_ndvi
Function that calculates the Normalised Diffference Vegetation Index (NDVI)

$ NDVI = \frac{NIR - Red}{NIR + Red} \in (-1, 1)$



In [16]:
def calculate_ndvi(band_dict):
  """
  Function that calculates the Normalised Diffference Vegetation Index (NDVI) from the multispectral bands of a satellite image

    parameter:  band_dict -> dictionary of multispectral band values
    return:     ndvi -> Normalised Diffference Vegetation Index (NDVI) value

  """
  try:
      # Extract the values for the NIR (Near Infrared) and Red bands from the dictionary
      nir = band_dict['B5']  # Assuming 'B5' is the NIR band
      red = band_dict['B4']  # Assuming 'B4' is the Red band

      # Calculate NDVI
      ndvi = (nir - red) / (nir + red)

      return ndvi

  except KeyError:
      return {"error": "Required bands (NIR and Red) not found in the band dictionary."}
  except ZeroDivisionError:
      return {"error": "Division by zero error. Check if NIR + Red is zero."}
  except Exception as e:
      return {"error": "An unexpected error occurred: " + str(e)}

# -----------------------------------------------------------------------------------------------------------------
# Example usage:
ndvi_value = calculate_ndvi(band_data)
print("NDVI:", ndvi_value)


NDVI: 0.17273035416054722


### calculate_vari
Function that calculates the Visible Atmospherically Resistant Index (VARI)

$VARI = \frac{Green - Red}{Green + Red - Blue}$

In [17]:
def calculate_vari(band_dict):
  """
  Function that calculates the Visible Atmospherically Resistant Index (VARI) from the multispectral bands of a satellite image

    parameter:  band_dict -> dictionary of multispectral band values
    return:     vari -> Visible Atmospherically Resistant Index (VARI) value

  """
  try:
      # Extract the values for the Blue, Red, and Green bands from the dictionary
      blue = band_dict['B2']  # Assuming 'B2' is the Blue band
      red = band_dict['B4']   # Assuming 'B4' is the Red band
      green = band_dict['B3'] # Assuming 'B3' is the Green band

      # Calculate VARI
      vari = (green - red) / (green + red - blue)

      return vari

  except KeyError:
      return {"error": "Required bands (Blue, Red, and Green) not found in the band dictionary."}
  except ZeroDivisionError:
      return {"error": "Division by zero error. Check if Green + Red - Blue is zero."}
  except Exception as e:
      return {"error": "An unexpected error occurred: " + str(e)}

# -----------------------------------------------------------------------------------------------------------------
# Example usage:
vari_result = calculate_vari(band_data)
print("VARI:", vari_result)


VARI: -0.06540748739282189


### calculate_savi
Function that calculates the Soil Adjusted Vegetation Index (SAVI)

$ SAVI = \frac{(1 + L) \cdot (NIR - Red)}{NIR + Red + L} \in (-1.0, 1.0)$

In [18]:
def calculate_savi(band_dict):
  """
  Function that calculates the Soil Adjusted Vegetation Index (SAVI) from the multispectral bands of a satellite image

    parameter:  band_dict -> dictionary of multispectral band values
    return:     savi -> Soil Adjusted Vegetation Index (SAVI) value

  """
  try:
      # Extract the values for the NIR (Near Infrared) and Red bands from the dictionary
      nir = band_dict['B5']  # Assuming 'B5' is the NIR band
      red = band_dict['B4']  # Assuming 'B4' is the Red band

      # Set the soil adjustment factor (L)
      L = 0.5

      # Calculate SAVI
      savi = ((1 + L) * (nir - red)) / (nir + red + L)

      return savi

  except KeyError:
      return {"error": "Required bands (NIR and Red) not found in the band dictionary."}
  except ZeroDivisionError:
      return {"error": "Division by zero error. Check if NIR + Red + L is zero."}
  except Exception as e:
      return {"error": "An unexpected error occurred: " + str(e)}

# -----------------------------------------------------------------------------------------------------------------
savi_result = calculate_savi(band_data)
print("SAVI:", savi_result)


SAVI: 0.11526018357934963


### calculate_ndwi

Function that calculates the Normalised Diffference Water Index (NDWI)

$ NDWI = \frac{Green - NIR}{Green + NIR} \in (-1, 1)$

In [19]:
def calculate_ndwi(band_dict):
  """
  Function that calculates the Normalised Diffference Water Index (NDWI) from the multispectral bands of a satellite image

    parameter:  band_dict -> dictionary of multispectral band values
    return:     ndwi -> Normalised Diffference Water Index (NDWI) value

  """
  try:
      # Extract the values for the Green and NIR bands from the dictionary
      green = band_dict['B3']  # Assuming 'B3' is the Green band
      nir = band_dict['B5']    # Assuming 'B5' is the NIR band

      # Calculate NDWI
      ndwi = (green - nir) / (green + nir)

      return ndwi

  except KeyError:
      return {"error": "Required bands (Green and NIR) not found in the band dictionary."}
  except ZeroDivisionError:
      return {"error": "Division by zero error. Check if Green + NIR is zero."}
  except Exception as e:
      return {"error": "An unexpected error occurred: " + str(e)}

# -----------------------------------------------------------------------------------------------------------------
# Example usage:
ndwi_value = calculate_ndwi(band_data)
print("NDWI:", ndwi_value)

NDWI: -0.2031022917735219


### calculate_evi

Function that calculates the Enhanced Vegetation Index (EVI)

$ EVI =  \frac{2.5 \cdot (NIR - Red)}{NIR + 6 \cdot Red - 7.5 \cdot Blue + 1} $

In [20]:
def calculate_evi(band_dict):
  """
  Function that calculates the Enhanced Vegetation Index (EVI) from the multispectral bands of a satellite image

    parameter:  band_dict -> dictionary of multispectral band values
    return:     evi -> Enhanced Vegetation Index (EVI) value

  """
  try:
      # Extract the values for the Blue, Red, and NIR bands from the dictionary
      blue = band_dict['B2']  # Assuming 'B2' is the Blue band
      red = band_dict['B4']   # Assuming 'B4' is the Red band
      nir = band_dict['B5']   # Assuming 'B5' is the NIR band

      # Parameters for EVI calculation
      G = 2.5
      C1 = 6.0
      C2 = 7.5
      L = 1.0  # Can be adjusted for different regions

      # Calculate EVI
      evi = G * (nir - red) / (nir + (C1 * red) - (C2 * blue) + L)

      return evi

  except KeyError:
      return {"error": "Required bands (Blue, Red, and NIR) not found in the band dictionary."}
  except ZeroDivisionError:
      return {"error": "Division by zero error. Check if NIR + C2*Red + L is zero."}
  except Exception as e:
      return {"error": "An unexpected error occurred: " + str(e)}

# -----------------------------------------------------------------------------------------------------------------
# Example usage:
evi_value = calculate_evi(band_data)
print("EVI:", evi_value)


# Equation Reference:
# https://developers.arcgis.com/documentation/arcgis-add-ins-and-automation/arcpy/
# https://www.usgs.gov/landsat-missions/landsat-enhanced-vegetation-index#:~:text=In%20Landsat%208%2D9%2C%20EVI,shown%20in%20the%20table%20below.

EVI: 0.17672246728768667


## Image Collection
Code was then developed to extrapolate functions to satellite image collection

### get_collection_landsat_image_ids
Function that extracts a list of Landsat 8 image IDs of a co-ordinate specified location within a date range

In [23]:
def get_collection_landsat_image_ids(start_date, end_date, latitude, longitude):
  """
  Function that extracts a list of Landsat 8 image IDs of a co-ordinate specified location within a date range

    parameter:  start_date -> start date of the date range
                end_date -> end date of the date range
                latitude -> latitude co-ordinate of desired location
                longitude -> longitude co-ordinate of desired location
    return:     image_id_list -> list of Landsat 8 image IDs

  """

  # Create a point of interest (POI) as a geometry
  poi = ee.Geometry.Point(longitude, latitude)

  # Create an image collection for Landsat imagery
  landsat_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
        .filterBounds(poi) \
        .filterDate(ee.Date.parse('YYYYMMdd', start_date), ee.Date.parse('YYYYMMdd', end_date))

  # Get a list of image IDs in the collection
  image_ids = landsat_collection.aggregate_array('system:id')

  # Get the image IDs as a Python list
  image_id_list = image_ids.getInfo()

  return image_id_list

# -----------------------------------------------------------------------------------------------------------------
# Example usage:
start_date = '20210101'
end_date = '20211231'
latitude = 37.7749
longitude = -122.4194

landsat_ids = get_collection_landsat_image_ids(start_date, end_date, latitude, longitude)

# Print the list of Landsat image IDs
print('Landsat Image IDs:')
for image_id in landsat_ids:
    print(image_id)


Landsat Image IDs:
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210116
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210201
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210217
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210305
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210321
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210406
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210422
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210508
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210524
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210609
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210625
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210711
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210727
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210812
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210828
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210913
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210929
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20211015
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20211031
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20211116
LANDSAT/LC08/C01/T1_TOA/LC08_044034_20211202
LANDSAT/LC08/C01/T1_TOA/LC08_044034_

In [1]:
# # POTENTIAL FUTURE DEVELOPMENT: EXTRACTS BOTH LANDSAT 7 AND LANDSAT 8

# # Define a function to get Landsat image IDs for Landsat 8 and 7
# def get_collection_landsat_image_ids(start_date, end_date, latitude, longitude):
#     # Create a point of interest (POI) as a geometry
#     poi = ee.Geometry.Point(longitude, latitude)

#     # Create separate image collections for Landsat 8 and Landsat 7
#     landsat_8_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
#         .filterBounds(poi) \
#         .filterDate(ee.Date.parse('YYYYMMdd', start_date), ee.Date.parse('YYYYMMdd', end_date))

#     landsat_7_collection = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA') \
#         .filterBounds(poi) \
#         .filterDate(ee.Date.parse('YYYYMMdd', start_date), ee.Date.parse('YYYYMMdd', end_date))

#     # Merge the two collections
#     landsat_collection = ee.ImageCollection(landsat_8_collection.merge(landsat_7_collection))

#     # Get a list of image IDs in the merged collection
#     image_ids = landsat_collection.aggregate_array('system:id')

#     # Get the image IDs as a Python list
#     image_id_list = image_ids.getInfo()

#     return image_id_list

# # Example usage:
# start_date = '20210101'
# end_date = '20211231'
# latitude = 37.7749  # Replace with your latitude
# longitude = -122.4194  # Replace with your longitude

# landsat_ids = get_collection_landsat_image_ids(start_date, end_date, latitude, longitude)

# # Print the list of Landsat image IDs
# print('Landsat Image IDs:')
# for image_id in landsat_ids:
#     print(image_id)

### create_df_from_image_ids
Function that creates a Pandas Dataframe from the Landsat 8 image collection: Image ID, Date, Co-ordinates, Band Values, Cloud Cover Percentage and Plant Indices.

In [24]:
def create_df_from_image_ids(image_ids, latitude, longitude):
  """
  Function that creates a Pandas Dataframe from the Landsat 8 image collection: Image ID, Date, Co-ordinates, Band Values, Cloud Cover Percentage and Plant Indices.

    parameter:  image_ids -> list of Landsat 8 image IDs
                latitude -> latitude co-ordinate of desired location
                longitude -> longitude co-ordinate of desired location
    return:     df -> Pandas DataFrame of satellite data information for each Landsat 8 image ID

  """
  data = []

  for image_id in image_ids:
      # Extract date from the image ID and remove hyphens
      date_str = image_id.split('_')[3]
      date = ''.join(date_str.split('-'))

      # Get Landsat band values for the current image
      band_values = get_landsat_bands(image_id, latitude, longitude)

      cloud = get_cloud_cover(image_id)

      # Calculate Plant Indices (NDVI, EVI, SAVI, NDWI, EVI) for the current image
      ndvi = calculate_ndvi(band_values)
      evi = calculate_evi(band_values)
      savi = calculate_savi(band_values)
      ndwi = calculate_ndwi(band_values)
      vari = calculate_vari(band_values)

      # Append the data as a dictionary to the list, including band values
      data.append({'Landsat Image ID': image_id, 'Date': date, 'Longitude': longitude, 'Latitude': latitude, **band_values, 'Cloud Cover': cloud, 'NDVI': ndvi, 'EVI': evi, 'SAVI': savi, 'VARI': vari, 'NDWI': ndwi})

  # Create the Pandas DataFrame using pandas.concat
  df = pd.concat([pd.DataFrame([d]) for d in data], ignore_index=True)

  return df

# -----------------------------------------------------------------------------------------------------------------
# Example usage
start_date = '20210101'  # Format: YYYYMMDD
end_date = '20211231'    # Format: YYYYMMDD
latitude = 37.7749  # Example latitude
longitude = -122.4194  # Example longitude

image_ids = get_collection_landsat_image_ids(start_date, end_date, latitude, longitude)
df = create_df_from_image_ids(image_ids, latitude, longitude)

# df.head()                      # Uncomment to view the individual Landsat 8 Satellite generated dataset
# print(df.info())               # Uncomment to view the Pandas DataFrame's characteristics (# columns, # rows, variables, variable types)

# save_df_to_drive(df, '/content/drive/My Drive/Colab Notebooks/Data/Flux Tower Data/Ameriflux/Test.csv')   #Uncommnet to save Pandas DataFrame as CSV file


,Landsat Image ID,Date,Longitude,Latitude,B1,B2,B3,B4,B5,B6,...,B9,B10,B11,BQA,Cloud Cover,NDVI,EVI,SAVI,VARI,NDWI
0,LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210116,20210116,-122.4194,37.7749,0.170105,0.143981,0.105994,0.096872,0.096230,0.094645,...,0.001456,289.041779,287.530060,2720,25.79,-0.003327,-0.002687,-0.001390,0.154909,0.048285
1,LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210201,20210201,-122.4194,37.7749,0.151685,0.126327,0.090686,0.076268,0.067377,0.066952,...,0.000541,287.479736,285.690094,2720,34.62,-0.061895,-0.038486,-0.020720,0.354900,0.147469
2,LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210217,20210217,-122.4194,37.7749,0.158330,0.135331,0.103563,0.094145,0.093394,0.088242,...,0.003958,286.675568,286.088440,2720,6.01,-0.004003,-0.002917,-0.001638,0.150985,0.051629
3,LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210305,20210305,-122.4194,37.7749,0.190147,0.171368,0.137787,0.140772,0.152287,0.135014,...,0.014288,291.385254,289.319427,2720,8.62,0.039292,0.040451,0.021780,-0.027840,-0.049984
4,LANDSAT/LC08/C01/T1_TOA/LC08_044034_20210321,20210321,-122.4194,37.7749,0.173140,0.159077,0.134845,0.141741,0.156751,0.152099,...,0.001920,294.518341,293.270386,2720,0.03,0.050285,0.046092,0.028197,-0.058688,-0.075125


## Ameriflux Application

Note: the code's commenting often refers to two attempts (where one will always be commented out and the other implemented). Attempt 1 refers to the spatially constricted 66 datasets and attempt 2 refers to the all inclusive 375 datasets. It is important that before running, each of the following cells are implementing the same attempt (i.e. all commented out or included sections agree on which attmept is being implemented).

### Ameriflux Site Overview Data

In [ ]:
# Ameriflux allows you to download a CSV file summarising the flux tower sites whose data you have chosen to use and their characteristics.
#     -   the importance of this is so link the flux tower site (Site ID) with its co-ordinate point

# Specify the Google Drive file path of the AmeriFlux Site Overview Dataset
# ------------------------------- ATTEMPT 1 -------------------------------
site_overview_file_path = '/content/drive/My Drive/Colab Notebooks/Data/Flux Tower Data/Ameriflux/AmeriFlux Site Description.csv'
# -------------------------------------------------------------------------

# ------------------------------- ATTEMPT 2 -------------------------------
# site_overview_file_path = '/content/drive/My Drive/Colab Notebooks/Data/Flux Tower Data/Ameriflux/AmeriFlux Site Description Extended.csv'
# -------------------------------------------------------------------------

# Load the AmeriFlux Site Overview Dataset into a Pandas DataFrame
df_site_overview = pd.read_csv(site_overview_file_path, delimiter=';')

# df_site_overview.head()                               # Uncomment to ensure proper csv file upload

# Filter the AmeriFlux Site Overview Dataset
df_site_overview['Years of AmeriFlux BASE Data'] = df_site_overview['Years of AmeriFlux BASE Data'].astype(str)                               # Converts list to string
df_site_overview['Start Year'] = df_site_overview['Years of AmeriFlux BASE Data'].apply(lambda x: min(map(int, x.strip('()').split(','))))    # Extracts the first year from the list
df_site_overview['End Year'] = df_site_overview['Years of AmeriFlux BASE Data'].apply(lambda x: max(map(int, x.strip('()').split(','))))      # Extracts the last year from the list

# df_site_overview.head()                               # Uncomment to ensure proper start and end year extraction

important_columns = ['Site ID', 'Latitude (degrees)', 'Longitude (degrees)', 'Start Year', 'End Year']    # Specifies important columns (the rest will be discarded for simplicity)
df_site_overview_filtered = df_site_overview[important_columns]                                           # Creates a new filtered Pandad DataFrame with only the important columns

# Saves the filtered AmeriFlux Site Overview Dataset the a specified Google Drive file path
# ------------------------------- ATTEMPT 1 -------------------------------
save_df_to_drive(df_site_overview_filtered, '/content/drive/My Drive/Colab Notebooks/Data/Flux Tower Data/Ameriflux/AmeriFlux Site Description Filtered.csv')
# -------------------------------------------------------------------------

# ------------------------------- ATTEMPT 2 -------------------------------
# save_df_to_drive(df_site_overview_filtered, '/content/drive/My Drive/Colab Notebooks/Data/Flux Tower Data/Ameriflux/AmeriFlux Site Description Extended Filtered.csv')
# -------------------------------------------------------------------------

# df_site_overview_filtered.head()                      # Uncomment to view simplified/ filtered database containing the needed variables
# print(df_site_overview_filtered.info())               # Uncomment to view the Pandas DataFrame's characteristics (# columns, # rows, variables, variable types)


An error occurred: name 'os' is not defined


,Site ID,Latitude (degrees),Longitude (degrees),Start Year,End Year
0,US-ASH,36.1697,-120.2010,2016,2017
1,US-ASL,36.9466,-120.1024,2016,2017
2,US-ASM,36.1777,-120.2026,2016,2017
3,US-Bi1,38.0992,-121.4993,2016,2023
4,US-Bi2,38.1091,-121.5351,2017,2023


### Ameriflux Individual Datasets

#### get_date_range_and_coordinates
Function that obtains the co-ordinates and operational date range of a flux tower from the AmeriFlux Site Overview dataset to be used as input parameters in extracting the landsat satellite image dataset


In [ ]:
def get_date_range_and_coordinates(site_overview_df, file_name):

  """
  Function that obtains the co-ordinates and operational date range of a flux tower from the AmeriFlux Site Overview dataset
    to be used as input parameters in extracting the landsat satellite image dataset

    parameter:  site_overview_df -> overview Pandas DataFrame describing the Ameriflux flux towers and their characteristics
                file_name -> file name of an individual Ameriflux flux tower dataset
    return:     data_array -> array of an individual Ameriflux flux tower's characteristics (co-ordinates and date range)

  """
  # Extract the Site ID from the flux tower dataset
  pattern = r'AMF_(.*?)_BASE'                 # Define a regex pattern to match the desired substring - in this case the Site ID of the flux tower
  match = re.search(pattern, file_name)       # Use re.search to find the match (i.e. the row in the overview table that refers to the name of the individual flux tower dataset)

  site_id = match.group(1)
  row_number = site_overview_df[site_overview_df['Site ID'] == site_id].index[0]      # Obtain the row number of the matched Site ID in the AmerFlux Site Overview dataset
  longitude = site_overview_df.loc[row_number, 'Longitude (degrees)']                 # Extract the longitude from the specified row
  latitude = site_overview_df.loc[row_number, 'Latitude (degrees)']                   # Extract the latitude from the specified row
  start_year = site_overview_df.loc[row_number, 'Start Year']                         # Extract the start year from the specified row
  end_year = site_overview_df.loc[row_number, 'End Year']                             # Extract the end year from the specified row

  data_array = [latitude, longitude, start_year, end_year]              # Append extracted parameters to a list

  return data_array

# -----------------------------------------------------------------------------------------------------------------
# Example usage

parameter_array = get_date_range_and_coordinates(df_site_overview_filtered, 'AMF_US-ASH_BASE_HH_1-5.csv')
print(parameter_array)

[36.1697, -120.201, 2016, 2017]


In [ ]:
# Extracting parameters from data array and using them to create the Landsat 8 Satellite dataset
latitude = parameter_array[0]
longitude = parameter_array[1]
start_year = str(parameter_array[2])
start_date = start_year+'0101'
end_year = str(parameter_array[3])
end_date = end_year+'0101'

landsat_ids = get_collection_landsat_image_ids(start_date, end_date, latitude, longitude)
landsat_df = create_df_from_image_ids(landsat_ids, latitude, longitude)

landsat_df.head()

,Landsat Image ID,Date,Longitude,Latitude,B1,B2,B3,B4,B5,B6,...,B9,B10,B11,BQA,Cloud Cover,NDVI,EVI,SAVI,VARI,NDWI
0,LANDSAT/LC08/C01/T1_TOA/LC08_042035_20160121,20160121,-120.201,36.1697,0.413524,0.403400,0.370163,0.383797,0.449826,0.323131,...,0.057237,268.326874,266.607117,6896,75.38,0.079208,0.227030,0.074268,-0.038891,-0.097152
1,LANDSAT/LC08/C01/T1_TOA/LC08_042035_20160206,20160206,-120.201,36.1697,0.148960,0.124333,0.097490,0.091097,0.181686,0.160547,...,0.001889,287.458771,286.753540,2720,2.22,0.332091,0.284593,0.175836,0.099491,-0.301587
2,LANDSAT/LC08/C01/T1_TOA/LC08_042035_20160222,20160222,-120.201,36.1697,0.165528,0.147591,0.130877,0.130748,0.272864,0.182339,...,0.001707,290.500214,289.658813,2720,8.35,0.352110,0.373823,0.235913,0.001130,-0.351679
3,LANDSAT/LC08/C01/T1_TOA/LC08_042035_20160309,20160309,-120.201,36.1697,0.136925,0.113631,0.098599,0.076108,0.281596,0.141400,...,0.007746,286.096252,284.521301,2720,1.45,0.574465,0.579813,0.359369,0.368248,-0.481325
4,LANDSAT/LC08/C01/T1_TOA/LC08_042035_20160325,20160325,-120.201,36.1697,0.123001,0.101011,0.089807,0.067193,0.324814,0.165968,...,0.001326,293.769165,292.400177,2720,8.39,0.657185,0.663703,0.433216,0.403900,-0.566798
